In [3]:
import pandas as pd
import joblib
import json
from io import StringIO
from sklearn.model_selection import train_test_split

import mlflow
import matplotlib.pyplot as plt
from mlflow import MlflowClient
import requests

In [47]:
# mlflow.set_tracking_uri(uri='http://127.0.0.1:5000')
mlflow.set_tracking_uri(uri='http://127.0.0.1:8000')
client = MlflowClient()
local_dir = ('C:/Users/Peter/DataspellProjects/pollock/')

In [64]:
data = pd.read_csv('data/original/Covid_data_full.csv')
# we use only part of data which was not used in training and validation
# data = data.iloc[900000:]
data = data.iloc[1047570:]




# let's make this classification task

data.loc[~data['DATE_DIED'].isin(['9999-99-99']), 'DATE_DIED'] = 1 #checks if the  column is not in the list ['9999-99-99'].
data.loc[data['DATE_DIED'] == '9999-99-99', 'DATE_DIED'] = 0


data

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
1047570,1,12,2,1,0,97,2,27,97,2,...,2,2,2,2,2,2,2,2,7,97
1047571,2,12,2,1,0,97,2,8,97,2,...,2,2,2,2,2,2,2,2,7,97
1047572,2,12,1,1,0,97,2,34,2,2,...,2,2,2,2,2,2,2,2,7,97
1047573,2,12,1,1,0,97,2,46,2,2,...,2,2,2,2,2,2,2,2,7,97
1047574,2,12,2,1,0,97,2,46,97,2,...,2,2,2,2,2,2,2,2,7,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,13,2,1,0,97,2,40,97,2,...,2,2,2,2,2,2,2,2,7,97
1048571,1,13,2,2,0,2,2,51,97,2,...,2,2,1,2,2,2,2,2,7,2
1048572,2,13,2,1,0,97,2,55,97,2,...,2,2,2,2,2,2,2,2,7,97
1048573,2,13,2,1,0,97,2,28,97,2,...,2,2,2,2,2,2,2,2,7,97


In [65]:
# X_request = data.drop('DATE_DIED',axis=1)
# y_request = data['DATE_DIED']

In [67]:
# X_request.to_csv('X_request.csv',index=False)
# y_request.to_csv('y_request.csv',index=False)

In [68]:
# #temp
model = joblib.load('model_knn.joblib')
y_pred = pd.DataFrame(model.predict(X))
y_pred.to_csv('y_pred_knn.csv', index=False)

##### 1. XGBoost

In [50]:


X_dict = X.reset_index().to_dict(orient='list')

predicted_data_response_xgb = requests.post('http://127.0.0.1:8000/model/predict_xgb',  json=X_dict) # local address
# predicted_data_response = requests.post('https://first-try-6o4cxe37ma-uc.a.run.app/model/predict',  json=X_test_dict) # docker addsess

print(predicted_data_response_xgb)
print(predicted_data_response_xgb.text)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
predicted_data_response_xgb.json()

In [ ]:
predicted_results = pd.read_json(StringIO(predicted_data_response_xgb.json())).set_index('index')
predicted_results

##### 2. Catboost

In [ ]:
predicted_data_response_catb = requests.post('http://127.0.0.1:8000/model/predict_catb',  json=X_dict) # local address
predicted_results_catb = pd.read_json(StringIO(predicted_data_response_catb.json())).set_index('index')
predicted_results_catb

##### 3. LightGBM

In [ ]:
predicted_data_response_lgbm = requests.post('http://127.0.0.1:8000/model/predict_lgbm',  json=X_dict) # local address
predicted_results_lgbm = pd.read_json(StringIO(predicted_data_response_lgbm.json())).set_index('index')
predicted_results_lgbm

##### 4. KNN

In [ ]:
predicted_data_response_knn = requests.post('http://127.0.0.1:8000/model/predict_knn',  json=X_dict) # local address
predicted_results_knn = pd.read_json(StringIO(predicted_data_response_knn.json())).set_index('index')
predicted_results_knn

##### 5. Logreg

In [ ]:
predicted_data_response_logreg = requests.post('http://127.0.0.1:8000/model/predict_logreg',  json=X_dict) # local address
predicted_results_logreg = pd.read_json(StringIO(predicted_data_response_logreg.json())).set_index('index')
predicted_results_logreg

##### 6. Naive Bayes


In [ ]:
predicted_data_response_nb = requests.post('http://127.0.0.1:8000/model/predict_nb',  json=X_dict) # local address
predicted_results_nb = pd.read_json(StringIO(predicted_data_response_nb.json())).set_index('index')
predicted_results_nb

##### 7. Ensemble

In [8]:
predicted_data_response_ens = requests.post('http://127.0.0.1:8000/model/predict_ens',  json=X_dict) # local address
predicted_results_ens = pd.read_json(StringIO(predicted_data_response_ens.json())).set_index('index')
predicted_results_ens

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /model/predict_ens (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D005B056C0>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение'))

### 3 Google Cloud Test

In [17]:


X_dict = X.reset_index().to_dict(orient='list')

# predicted_data_response_xgb = requests.post('http://127.0.0.1:8000/model/predict_xgb',  json=X_dict) # local address
predicted_data_response = requests.post('https://first-try-6o4cxe37ma-uc.a.run.app/model/predict',  json=X_test_dict) # docker address


print(predicted_data_response_xgb)
print(predicted_data_response_xgb.text)

NameError: name 'X_test_dict' is not defined

In [39]:
X_dict = X.reset_index().to_dict(orient='list')
predicted_data_response_xgb = requests.post('https://covidapp-bmvhwrbbeq-lm.a.run.app/model/predict_xgb',  json=X_dict) # docker address

In [24]:
print(predicted_data_response_xgb)
print(predicted_data_response_xgb.text)

<Response [200]>
"{\"index\":{\"0\":0,\"1\":1,\"2\":2,\"3\":3,\"4\":4},\"0\":{\"0\":0,\"1\":0,\"2\":0,\"3\":0,\"4\":0}}"


In [25]:
predicted_results = pd.read_json(StringIO(predicted_data_response_xgb.json())).set_index('index')
predicted_results

,0
index,
0,0
1,0
2,0
3,0
4,0


##### 2. Catboost

In [ ]:
data

In [32]:
X = pd.read_csv('X.csv')

In [34]:
X

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,1,12,2,2,2,1,54,97,2,2,2,2,2,1,2,2,2,1,7,2
1,2,12,1,1,97,2,46,2,2,1,2,2,2,2,2,2,2,2,7,97
2,2,3,1,1,97,2,51,2,2,2,2,2,2,2,2,2,2,2,7,97
3,2,4,1,2,2,2,45,2,2,2,2,2,2,2,2,2,2,2,3,2
4,2,12,1,1,97,2,38,2,2,2,2,2,2,2,2,2,2,2,3,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269995,2,12,1,1,97,2,50,2,2,2,2,2,1,2,2,1,2,2,7,97
269996,2,6,1,1,97,2,73,2,1,2,2,2,1,2,2,1,1,2,6,97
269997,2,12,2,1,97,2,29,97,2,2,2,2,2,2,2,2,2,2,3,97
269998,1,12,1,2,1,1,53,2,2,2,2,2,2,2,2,2,2,2,3,1


In [46]:
X_dict = X.reset_index().to_dict(orient='list')
predicted_data_response_catb = requests.post('https://covidapp-bmvhwrbbeq-lm.a.run.app/model/predict_catb',  json=X_dict) # local address
predicted_results_catb = pd.read_json(StringIO(predicted_data_response_catb.json())).set_index('index')
predicted_results_catb

,0
index,
0,1
1,0
2,0
3,0
4,0
...,...
11000,0
11001,0
11002,0


##### 3. LGBM

In [27]:
predicted_data_response_lgbm = requests.post('https://covidapp-bmvhwrbbeq-lm.a.run.app/model/predict_lgbm',  json=X_dict) # local address
predicted_results_lgbm = pd.read_json(StringIO(predicted_data_response_lgbm.json())).set_index('index')
predicted_results_lgbm

,0
index,
0,0
1,0
2,0
3,0
4,0


##### 4. KNN

In [28]:
predicted_data_response_knn = requests.post('https://covidapp-bmvhwrbbeq-lm.a.run.app/model/predict_knn',  json=X_dict) # local address
predicted_results_knn = pd.read_json(StringIO(predicted_data_response_knn.json())).set_index('index')
predicted_results_knn

,0
index,
0,0
1,0
2,0
3,0
4,0


##### 5. Logreg

In [29]:
predicted_data_response_logreg = requests.post('https://covidapp-bmvhwrbbeq-lm.a.run.app/model/predict_logreg',  json=X_dict) # local address
predicted_results_logreg = pd.read_json(StringIO(predicted_data_response_logreg.json())).set_index('index')
predicted_results_logreg

,0
index,
0,0
1,0
2,0
3,0
4,0


##### 6. Naive Bayes


In [30]:
predicted_data_response_nb = requests.post('https://covidapp-bmvhwrbbeq-lm.a.run.app/model/predict_nb',  json=X_dict) # local address
predicted_results_nb = pd.read_json(StringIO(predicted_data_response_nb.json())).set_index('index')
predicted_results_nb

,0
index,
0,0
1,0
2,0
3,0
4,0


##### 7. Ensemble

In [31]:
predicted_data_response_ens = requests.post('https://covidapp-bmvhwrbbeq-lm.a.run.app/model/predict_ens',  json=X_dict) # local address
predicted_results_ens = pd.read_json(StringIO(predicted_data_response_ens.json())).set_index('index')
predicted_results_ens

,0
index,
0,0
1,0
2,0
3,0
4,0
